In [2]:
import pandas as pd
from selenium import webdriver
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import os
from selenium.webdriver.common.keys import Keys
from time import sleep
from datetime import datetime

In [37]:
start_time = datetime.now()

#Input file path
filepath = r"C:\Users\anushrees1\OneDrive - KPMG\Documents\WC\France\Python Webscrapping\PlusSimple\Input\Address_Lat_Long.xlsx"
sheetname = "Sheet1"
df = pd.read_excel(filepath,sheet_name=sheetname)
driver = webdriver.Chrome(r"C:\Users\anushrees1\OneDrive - KPMG\Desktop\chromedriver.exe")
url='https://www.google.com/'

#output dataframe
data=pd.DataFrame(columns=['Address','Code Postal','Latitude','Longitude'])

for index, row in df.iterrows():
    try:
        #print(row['Numero et voie'])
        driver.get(url)
        search = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
        search.clear()
        search.send_keys(row['Numero et voie'])
        search.send_keys(Keys.RETURN) 
        sleep(0.2)
        wait = WebDriverWait(driver, 10)

        #finding search result class
        elems = driver.find_elements_by_xpath(".//*[starts-with(@class, 'C8TUKc rllt__link a-no-hover-decoration')]")
        if len(elems)>0:
            #print("Name:"+elems[0].text.split("\n")[0])
            elems[0].click()
            sleep(3)
            details=driver.find_element_by_xpath('//*[@class="zloOqf PZPZlf"]').text.split("\n")
            postalcode=str(int(row['Code postal']))
            if len([idx for idx, s in enumerate(details) if postalcode in s])!=0:
                url_link= driver.current_url
                print(url_link)
                exp = re.search(r"(\[[-]?\d+\.\d+\,[-]?\d+\.\d+\])", url_link)
                coords = re.sub("\[","",exp.group(0))
                coords = re.sub("\]","",coords)
                coords = coords.split(",")
                lat = coords[0]
                #print(lat)
                long = coords[1]
                #print(long)
            else:
                continue
            data = data.append({'Address':row['Numero et voie'],'Code Postal':row['Code postal'],'Latitude':lat,
                                  'Longitude':long}, ignore_index=True)
        elif len(elems)==0:
            # fetching clickable image class and enabling click
            elements=driver.find_elements_by_xpath(".//*[starts-with(@class, 'GosL7d cYnjBd')]")
            elements[0].click()
            sleep(5)
            url_link = driver.current_url
            exp = re.search(r"(\@[-]?\d+\.\d+\,[-]?\d+\.\d+\,)",url_link)
            coords = re.sub("\@","",exp.group(0))
            coords = re.sub("\,"," ",coords)
            coords = coords.split(" ")
            lat = coords[0]
            #print(lat)
            long = coords[1]
            #print(long)
            data = data.append({'Address':row['Numero et voie'],'Code Postal':row['Code postal'],'Latitude':lat,
                                  'Longitude':long}, ignore_index=True)                

        else :
            continue
    except:
        continue

driver.close()
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))


ZAC DES PLATIERES 71000
3
Name:Hôtel Kyriad Mâcon Sancé
['Address: ZAC des Platières, Sancé, 3 Rue du 19 Mars 1962, 71000 Mâcon, France']
71000
https://www.google.com/search?q=ZAC%20DES%20PLATIERES%2071000&source=hp&ei=a7JmYZ2QG5PM1sQPnKiRgAg&iflsig=ALs-wAMAAAAAYWbAeyLQnkICCgV-7aCQ0EL9AyqxCNyc&ved=2ahUKEwiXzInBlMfzAhX7xzgGHUwzBSAQvS56BAgGEDQ&uact=5&oq=ZAC+DES+PLATIERES+71000&gs_lcp=Cgdnd3Mtd2l6EANQhgpYmQtg3QtoAHAAeACAAQCIAQCSAQCYAQCgAQE&sclient=gws-wiz&tbs=lf:1,lf_ui:2&tbm=lcl&rflfq=1&num=10&rldimm=11198451823516631044&lqi=ChdaQUMgREVTIFBMQVRJRVJFUyA3MTAwMEj-ha_G5YCAgAhaJxAAEAEQAhgAGAEYAhgDIhd6YWMgZGVzIHBsYXRpZXJlcyA3MTAwMJIBBWhvdGVsqgEZEAEqFSIRemFjIGRlcyBwbGF0aWVyZXMoAA&rlst=f#rlfi=hd:;si:11198451823516631044,l,ChdaQUMgREVTIFBMQVRJRVJFUyA3MTAwMEj-ha_G5YCAgAhaJxAAEAEQAhgAGAEYAhgDIhd6YWMgZGVzIHBsYXRpZXJlcyA3MTAwMJIBBWhvdGVsqgEZEAEqFSIRemFjIGRlcyBwbGF0aWVyZXMoAA;mv:[[46.3370557,4.8463772],[46.3310361,4.8435181]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,

In [38]:
data

,Address,Code Postal,Latitude,Longitude
0,ZAC DES PLATIERES 71000,71000.0,46.3370557,4.8463772
1,ZNE INDUS DE KERPONT 56600,56600.0,47.7915109,-3.3184869
2,1 ROND POINT DU VIET NAM HEROIQUE 97200,97200.0,14.611924,-61.080794
3,ROND POINT MARCEL SIMON 54000,54000.0,48.7010632,6.1750385
4,LES ALPAGES DE CHANTEL 2 73700,73700.0,45.576223299999995,6.7861446999999995
5,4846 ROUTE DE LA CRAU RAPHELE 13280,13280.0,43.6476595,4.7060163
6,ROUTE DEPARTEMENTALE 1083 67640,67640.0,46.0969404,5.1221995
7,ROUTE ROUTE NATIONALE 109 34725,34725.0,43.6511252,3.5089482
8,2 RUE LORD CLIVE 34120,34120.0,43.4522258,3.4168945
9,VIALE LUIGI BODIO 37 A,NaN,45.496712,9.1655843


In [39]:
data.to_excel(r"C:\Users\anushrees1\OneDrive - KPMG\Documents\WC\France\Python Webscrapping\LatLong2.xlsx")

In [30]:
url_link

'https://www.google.com/maps/place/Rdpt+du+Viet-Nam+H%C3%A9ro%C3%AFque,+Fort-de-France+97200,+Martinique/data=!4m2!3m1!1s0x8c6aa6e3a3e7fccf:0xbf1adb6f022ab0d2?sa=X&ved=2ahUKEwjV9bHakcfzAhXqyTgGHQo5AIUQ8gF6BAgFEAE'